### Imports

In [ ]:
import numpy as np
import pandas as pd
import importlib
import matplotlib.pyplot as plt
import matplotlib
from pathlib import Path
import os
import string
import random
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, ConstantKernel as C
import pandas as pd
import julian
import datetime
import sys

sys.path.insert(0, '..')

### Data Download

In [ ]:
#HE0435
CosmoGrail = pd.read_table('../data/cosmograil/HE0435_Bonvin2016.rdb_.txt')

t = CosmoGrail['mhjd'].values[1:].astype('float')
A = CosmoGrail['mag_A'].values[1:].astype('float')
dA = CosmoGrail['magerr_A'].values[1:].astype('float')
B = CosmoGrail['mag_B'].values[1:].astype('float')
dB = CosmoGrail['magerr_B'].values[1:].astype('float')
C_ = CosmoGrail['mag_C'].values[1:].astype('float')
dC_ = CosmoGrail['magerr_C'].values[1:].astype('float')
D = CosmoGrail['mag_D'].values[1:].astype('float')
dD = CosmoGrail['magerr_D'].values[1:].astype('float')

### Print some light curve pairs

In [ ]:
%matplotlib notebook
plt.figure(figsize=(20,10))
plt.plot(t,A,'x', label = 'A')
plt.plot(t,B,'x', label = 'B')
plt.plot(t,C_, 'o', label = 'C')
plt.plot(t,D, 'o', label = 'D')
plt.gca().invert_yaxis()
plt.legend()
plt.show()

### GP Regression
In the regression difference method, a Gaussian process regression is performed on each quasar image light curve independently. The regressions are then shifted in time and subtracted
pair-wise, resulting in one difference curve for each pair of light
curves and its associated uncertainties envelope. The algorithm
minimises the variability of the difference curves by varying the
time shift.

In practice, for each of the n light curves, we evaluate the
GPR every 0.2 days. Given some trial time shifts, we express
the n (n − 1)/2 difference curves by subtracting linearly interpolated magnitudes of the shifted regression curves. Indeed, each
pair of curves has to be considered only once; for the variability


In [ ]:
kernel = C(0.1, (1e-3, 1e2)) * RBF(200, (1e-1, 5e2))
kernel2 = C(2, (1e-3, 1e2)) * Matern(length_scale=200.0, length_scale_bounds=(1, 300), nu=1.5)
gp1 = GaussianProcessRegressor(kernel=kernel2, alpha=dA**2,
                              n_restarts_optimizer=10, optimizer = 'fmin_l_bfgs_b', normalize_y =True)
gp2 = GaussianProcessRegressor(kernel=kernel2, alpha=dB**2,
                              n_restarts_optimizer=10, optimizer = 'fmin_l_bfgs_b', normalize_y =True)


gp1.fit(np.expand_dims(t,1), A)
gp2.fit(np.expand_dims(t,1), B)

In [ ]:
print(gp1.log_marginal_likelihood_value_)
print(gp2.log_marginal_likelihood_value_)
gp1.get_params()

In [ ]:
gp_step = 0.2
dt_ext = 100
support = np.arange(t[0] - dt_ext, t[-1] + dt_ext, gp_step)
y_pred1, sigma1 = gp1.predict(np.expand_dims(support,1), return_std=True)
y_pred2, sigma2 = gp2.predict(np.expand_dims(support,1), return_std=True)

### Plot Regression results

In [ ]:
%matplotlib notebook
plt.figure(figsize=(10,10))
plt.plot(np.expand_dims(t,1), B, 'g.', markersize=3, label='ObservationsB')
plt.plot(np.expand_dims(t,1), A, 'b.', markersize=3, label='ObservationsA')
plt.plot(np.expand_dims(support,1), y_pred1, 'b-', label='PredictionA')
plt.plot(np.expand_dims(support,1), y_pred2, 'g-', label='PredictionB')
plt.fill(np.concatenate([np.expand_dims(support,1), np.expand_dims(support,1)[::-1]]),
         np.concatenate([y_pred1 - 1.9600 * sigma1,
                        (y_pred1 + 1.9600 * sigma1)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval B')
plt.fill(np.concatenate([np.expand_dims(support,1), np.expand_dims(support,1)[::-1]]),
         np.concatenate([y_pred2 - 1.9600 * sigma2,
                        (y_pred2 + 1.9600 * sigma2)[::-1]]),
         alpha=.5, fc='g', ec='None', label='95% confidence interval A')
plt.xlabel('$x$')
plt.ylabel('$f(x)$')
plt.legend(loc='upper left')
plt.gca().invert_yaxis()

### Plot shifted curves and difference

In [ ]:
%matplotlib notebook
win = 1000
shift = 0
plt.plot(np.expand_dims(support,1)[win:-win],y_pred1[win:-win], color = 'g', label = 'A')
plt.plot(np.expand_dims(support,1)[win:-win],y_pred2[win+shift:-win+shift], color = 'b', label = 'B')
plt.legend()
plt.gca().invert_yaxis()

In [ ]:
%matplotlib notebook
win = 1000
shift = 0
diff = y_pred1[win:-win]-y_pred2[win+shift:-win+shift]
sigma_diff = sigma1[win:-win] + sigma2[win+shift:-win+shift]
plt.plot(np.expand_dims(support,1)[win:-win],diff, color = 'b', label = 'Difference')
plt.fill(np.concatenate([np.expand_dims(support,1)[win:-win], np.expand_dims(support,1)[win:-win][::-1]]),
         np.concatenate([diff - 1.9600 * sigma_diff,
                        (diff + 1.9600 * sigma_diff)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval difference')
plt.legend()
plt.gca().invert_yaxis()

### Loss Function

In [ ]:
%matplotlib notebook

def numeric_derivative(f, step):
    return (f[:-1] - f[1:])/step

def stencil_derivative(f, step):
    df = (f[:-4] - 8*f[1:-3] + 8*f[3:-1] - f[4:] )/(12*step)
    return df

def WAV(f, sigma, step):
    # classic finite difference derivative
    # f_prime = numeric_derivative(f, step)
    # weights = 2/(sigma[:-1]+sigma[1:])
    
    # stencil 5 points derivative
    f_prime = stencil_derivative(f, step)
    weights = 2/(sigma[2:-2]+sigma[2:-2])
    
    WAV = np.dot(np.abs(f_prime), weights)/np.sum(weights)
    return WAV

WAV(diff,sigma_diff,gp_step)

### Simple Test
Simple Grid-search. Better results can be probably obtained with an out-of-the-box nonlinear optimizer

In [ ]:
win = int(dt_ext/gp_step)
dt_min = -100
dt_max = +100
shift = np.arange(int(dt_min/gp_step),int(dt_max/gp_step),1)
WAV_values = []
for i in shift:
    diff = y_pred1[win:-win]-y_pred2[win+i:-win+i]
    sigma_diff = sigma1[win:-win] + sigma2[win+i:-win+i]
    WAV_values.append(WAV(diff,sigma_diff,gp_step))

estimated_delay = shift[np.argmin(WAV_values)] * gp_step    

print('Estimated delay: ', estimated_delay)

In [ ]:
plt.plot(shift*gp_step, WAV_values)
plt.xlabel('time lag[days]')
plt.ylabel('WAV')
plt.title('WAV loss function for HE0435 A vs B')
plt.savefig('../aux/outputs/wav_data_HE0435.pdf')

### Summary

In [ ]:
#BD ------> -6.80    (Milton2020: -5.4 ± 0.8)  (Courbin2011: -6.5 ± 0.7) 
#AD ------> -13.2   (Milton2020: -13.8 ± 0.9)  (Courbin2011: -14.9 ± 2.1) 
#AB ------> -8.4    (Milton2020: -9.0 ± 0.8)  (Courbin2011: -8.4 ± 2.1) 
#AC ------>  0.2  (Milton2020: -0.8 ± 0.7)  (Courbin2011: 0.6 ± 2.3) 
#BC ------>  +8.0   (Milton2020: +7.8 ± 0.9)  (Courbin2011: +7.8 ± 0.8) 
#CD ------>  -13.2   (Milton2020: -13.2 ± 0.8)  (Courbin2011: -14.3 ± 0.8)